In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data=pd.read_csv('/content/mobile_price_classification.csv')

In [3]:
data.isna().sum()

,0
battery_power,0
bluetooth,0
clock_speed,0
dual_sim,0
front_cam,0
4G,0
int_memory,0
m_dep,0
mobile_wt,0
n_cores,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   battery_power   2000 non-null   int64  
 1   bluetooth       2000 non-null   int64  
 2   clock_speed     2000 non-null   float64
 3   dual_sim        2000 non-null   int64  
 4   front_cam       2000 non-null   int64  
 5   4G              2000 non-null   int64  
 6   int_memory      2000 non-null   int64  
 7   m_dep           2000 non-null   float64
 8   mobile_wt       2000 non-null   int64  
 9   n_cores         2000 non-null   int64  
 10  primary_camera  2000 non-null   int64  
 11  px_height       2000 non-null   int64  
 12  px_width        2000 non-null   int64  
 13  ram             2000 non-null   int64  
 14  sc_h            2000 non-null   int64  
 15  sc_w            2000 non-null   int64  
 16  talk_time       2000 non-null   int64  
 17  three_g         2000 non-null   i

In [5]:
data.shape

(2000, 21)

In [6]:
! pip install tensorflow keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.7 MB/s eta 0:00:00


In [7]:
x = data.iloc[:,0:20]
y = data.iloc[:,20]

In [8]:
x

,battery_power,bluetooth,clock_speed,dual_sim,front_cam,4G,int_memory,m_dep,mobile_wt,n_cores,primary_camera,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0
2,563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0
3,615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0
4,1821,1,1.2,0,13,1,44,0.6,141,2,14,1208,1212,1411,8,2,15,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,794,1,0.5,1,0,1,2,0.8,106,6,14,1222,1890,668,13,4,19,1,1,0
1996,1965,1,2.6,1,0,0,39,0.2,187,4,3,915,1965,2032,11,10,16,1,1,1
1997,1911,0,0.9,1,1,1,36,0.7,108,8,3,868,1632,3057,9,1,5,1,1,0
1998,1512,0,0.9,0,4,1,46,0.1,145,5,5,336,670,869,18,10,19,1,1,1


In [9]:
y

,price_range
0,1
1,2
2,2
3,2
4,1
...,...
1995,0
1996,2
1997,3
1998,0


In [10]:
y.value_counts()

,count
price_range,
1,500
2,500
3,500
0,500


In [14]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [13]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import keras_tuner as kt


In [16]:
def build_model(hp):
  model = Sequential()

  model.add(Dense(hp.Int('units',min_value=16,max_value=128,step=16),activation='relu'))

  for i in range(hp.Int('num_layers',1,3)):
    model.add(Dense(hp.Int(f'units_{i}',min_value=16,max_value=128,step=16),activation='relu'))

  model.add(Dense(1,activation='sigmoid'))

  model.compile(
      optimizer = tf.keras.optimizers.get({
          'class_name': hp.Choice('optimizer',values=['adam','rmsprop','sgd']),
          'config' : {'learning_rate':hp.Float('learning_rate',min_value=1e-4,max_value=1e-2,sampling= 'log')}
      }),
      loss = 'binary_crossentropy',
      metrics = ['accuracy']
    )
  return model

In [20]:
tuner = kt.RandomSearch(
    build_model,
    objective = 'val_accuracy',
    max_trials = 10
)

Reloading Tuner from ./untitled_project/tuner0.json


In [21]:
tuner.search(
    x_train,y_train,
    epochs =20,
    validation_split = 0.2,
    verbose = 1
)

Trial 10 Complete [00h 00m 05s]
val_accuracy: 0.24375000596046448

Best val_accuracy So Far: 0.25312501192092896
Total elapsed time: 00h 04m 14s


In [22]:
best_hps = tuner.get_best_hyperparameters(1)[0]
print('Units in 1st layer',best_hps.get('units'))
print('No of layers',best_hps.get('num_layers'))
for i in range(best_hps.get('num_layers')):
  print(f"units in layer {i+1}:{best_hps.get(f'units_{i}')}")
print('optimizer :',best_hps.get('optimizer'))
print('Learning rate :',best_hps.get('learning_rate'))

Units in 1st layer 112
No of layers 1
units in layer 1:48
optimizer : rmsprop
Learning rate : 0.0022855349849264612


In [23]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2

In [24]:
def build_model(hp):
    model = Sequential()

    model.add(Dense(
        units=hp.Int('units', min_value=32, max_value=128, step=32),
        activation='relu',
        kernel_regularizer=l2(hp.Float('l2_regularizer', min_value=1e-5, max_value=1e-2, sampling='log')),
    ))
    model.add(Dropout(hp.Float('dropout_rate', min_value=0.2, max_value=0.5, step=0.1)))

    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [25]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    directory='keras_tuner',
    project_name='tune_dropout_regularization'
)

In [27]:
tuner.search(
    x_train, y_train,
    epochs=20,
    validation_split=0.2,
    verbose=1
)

Trial 5 Complete [00h 00m 05s]
val_accuracy: 0.25312501192092896

Best val_accuracy So Far: 0.25312501192092896
Total elapsed time: 00h 00m 31s


In [28]:
best_hps = tuner.get_best_hyperparameters(1)[0]
print('Units in 1st layer',best_hps.get('units'))
print('L2 Regularizer :',best_hps.get('l2_regularizer'))
print("Dropout rate :", best_hps.get('dropout_rate'))

Units in 1st layer 128
L2 Regularizer : 0.00017801479753512488
Dropout rate : 0.4
